# Use `PyRosetta` to score monomeric designs with an array of structure- and sequence-based scoring metrics

The goal of this notebook is to give an example of a computational pipeline to analyze monomeric protein designs with `PyRosetta` with a variety of biophysical metrics, such as the number of exposed hydrophobic residues or the degree of shape complementarity between elements of secondary structure.

As an example, I run this pipeline on a subset of 10 designs from [Rocklin et al., 2017, Science](http://science.sciencemag.org/content/357/6347/168), which was the initial study that used the protein-stability assay to quantitatively analyze the correlates of successful protein design.

Then, as a reality check that the pipeline is working, I compare the scores computed in this pipeline with the scores from the Rocklin et al. study.

## Import `Python` modules and `RosettaScripts` XML files and initialize directories

First, I import various `Python` modules, including `PyRosetta`. I will also initialize a directory to store the results of the analysis (`./results/`).

In [1]:
import os
import sys
sys.path.append('scripts/')
import glob
import pandas
import re
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
import numpy
import math

# Import and initialize PyRosetta
import pyrosetta
pyrosetta.init()

# Initialize a results directory
resultsdir = './results/'
if not os.path.isdir(resultsdir):
    os.makedirs(resultsdir)

Found rosetta database at: /software/pyrosetta2/latest/setup/pyrosetta/database; using it....
PyRosetta-4 2017 [Rosetta PyRosetta4.Release.python27.ubuntu 2017.50+release.34adaaf34adaaf5b1bb06259ee33056a854849cbeaf2201 2017-12-13T10:25:37] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions.
Created in JHU by Sergey Lyskov and PyRosetta Team.


core.init: Rosetta version: PyRosetta4.Release.python27.ubuntu r162 2017.50+release.34adaaf 34adaaf5b1bb06259ee33056a854849cbeaf2201 http://www.pyrosetta.org 2017-12-13T10:25:37
core.init: command: PyRosetta -ex1 -ex2aro -database /software/pyrosetta2/latest/setup/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=-923907081 seed_offset=0 real_seed=-923907081
core.init.random: RandomGenerator:init: Normal mode, seed=-923907081 RG_type=mt19937


Next, I import the scoring protocols. The exact scoring protocols are encoded in a series of `RosettaScripts` XML files (see [this tutorial](https://www.rosettacommons.org/demos/latest/tutorials/scripting_with_rosettascripts/scripting_with_rosettascripts) if you would like more information on how these are encoded). These files are stored in the directory `./scripts/xmls/` with one XML per scoring metric. The below import reads in the contents of each file as a dictionary of the form `{metric name : string of script contents}`.

In [2]:
# import `analysis_scripts`, which is a dictionary
# of the form {metric name : script contents}
from xmls import analysis_scripts

## Initialize a score function and the XML-encoded scoring metrics in `PyRosetta`

First, I initialize a `Rosetta` score function, which is the biophysical model used to score designs based on their sequence and structure. The score function has been updated many times, meaning there are multiple different versions. We will initialize the most current variant, `beta`.

In [3]:
# Initialize a specific score function
scorefxn = pyrosetta.rosetta.core.scoring.ScoreFunctionFactory.create_score_function("beta")

core.scoring.ScoreFunctionFactory: [ WARNING ] **************************************************************************
*****************************************************
****************************************************
beta may be a 'beta' scorefunction, but ScoreFunctionFactory thinks the beta flags weren't set.  Your scorefunction may be garbage!
**************************************************************************
*****************************************************
****************************************************
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv

Next, I initilize each of the XML-encoded scoring metrics in `PyRosetta` so that they can be used downstream to actually score the designs.

In [5]:
# Initialize the filters by reading in and processing the XML file for each filter
pr_filters = {
    
    filter_name : pyrosetta.rosetta.protocols.rosetta_scripts.XmlObjects.create_from_string(script_contents)
        .get_filter(filter_name)
    for (filter_name, script_contents) in analysis_scripts.items()
    if filter_name in ['unsat_hbond', 'new_buns_sc_heavy']
    
}

protocols.rosetta_scripts.RosettaScriptsParser: Generating XML Schema for rosetta_scripts...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Initializing schema validator...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Validating input script...
protocols.rosetta_scripts.RosettaScriptsParser: ...done
protocols.rosetta_scripts.RosettaScriptsParser: Parsed script:
<ROSETTASCRIPTS>
	<MOVERS/>
	<FILTERS>
		<BuriedUnsatHbonds confidence="0" jump_number="0" name="unsat_hbond"/>
	</FILTERS>
	<PROTOCOLS>
		<Add filter_name="unsat_hbond"/>
	</PROTOCOLS>
</ROSETTASCRIPTS>
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline

UnicodeDecodeError: 'ascii' codec can't decode byte 0xe2 in position 694: ordinal not in range(128)

## Score the input PDB files

First I define a function, `score_pdb_file`, which uses `PyRosetta` to score an input PDB file with the above metrics.

Note: You may want to parallelize the scoring step if you're running it on thousands of designs. With this in mind, I wrote `score_pdb_file` to accept a single PDB file so that it could be called many times in parallel. Let me know if you would like suggestions for how to parallelize this task. I have been using [`Jug`](https://jug.readthedocs.io/en/latest/) to do so, and would be happy to describe my strategy in greater detail if you would like.

In [ ]:
def score_pdb_file(pdb_file_name):
    """
    Score an input PDB file using an array of Rosetta-based metrics
    
    PyRosetta computes these scores using the metrics encoded in the
    RosettaScripts XML files in the directory `./scripts/xmls/`.
    
    Args:
        `pdb_file_name` : The path to a PDB file (string)
    
    Returns:
        A dataframe with scores for the PDB file
    """
    
    # Read in the PDB file and convert it to a pose
    pose = pyrosetta.pose_from_pdb(pdb_file_name)
    
    # Compute scores from energies from the score function and store
    # the scores in a dictionary with the form {score name : value}
    scores_dict = {}
    scorefxn(pose)
    for score in scorefxn.get_nonzero_weighted_scoretypes():
        re_pattern = re.compile(r'ScoreType\.(?P<score_name>\w+)')
        score_name = re_pattern.search(str(score)).group('score_name')
        scores_dict[score_name] = pose.energies().total_energies()[score]
    
    # Compute XML-based scores and add these scores to the dictionary
    # with all the scores
    for (filter_name, pr_filter) in pr_filters.items():
        scores_dict[filter_name] = pr_filter.score(pose)
    
    # Conver the dictionary to a dataframe with the PDB file name as an index
    scores_df = pandas.DataFrame.from_dict({pdb_file_name : scores_dict}, orient='index')
    
    return scores_df

Next, I read in a test set of PDB files from the Rocklin et al. study, which are stored in the directory `./data/rocklin_rd4_examples/`. I then score each of them using the above function (this step could be parallelized). Finally, I concatenate the data into a single dataframe.

In [ ]:
# Get the input PDB files
input_pdb_files = glob.glob('./data/rocklin_rd4_examples/*.pdb')

# Score each PDB file one at a time
scores_dfs = [score_pdb_file(f) for f in input_pdb_files]

# Concatenate each of the PDB-specific dataframes
test_scores = pandas.concat(scores_dfs)

Below, I show the scores as a dataframe.

In [ ]:
# Show the dataframe
test_scores['PDB_name'] = test_scores.index.map(lambda x: os.path.basename(x))
test_scores.set_index('PDB_name', inplace=True)
test_scores_outfile = os.path.join(resultsdir, 'scores.csv')
test_scores.to_csv(test_scores_outfile, index_label='PDB_name')
test_scores

## Compare the scores computed above to the scores computed in the Rocklin et al. study, just to make sure our scoring pipeline gives similar results

First, I read in scores for these structures taken from the Rocklin et al. study.

In [ ]:
rocklin_scores_f = './data/Rocklin_rd4_relax_scored_filtered_betanov15.sc'
rocklin_scores = pandas.read_csv('./data/Rocklin_rd4_relax_scored_filtered_betanov15.sc', sep='\t')
rocklin_scores.set_index('description', inplace=True)
del rocklin_scores['SCORE:'] # non-informative line, is 'SCORE:' in all entries

Next, I compute the correlation between our scores and the Rockclin scores for each metric. Almost all metrics are highly correlated with the Rocklin et al. study. A few metrics could not be correlated (i.e., R = `NaN`). As shown in the below correlation plots, that is because all designs have the exact same score in both scoring protocols, so there is no spead in the data.

In [ ]:
# Get an ordered list of design PDB file names to compare
pdbs = list(test_scores.index.values)

# Compute the correlation between our scores and the Rocklin scores for each design
corr_d = {}
corr_cutoff = 0.95
metrics_below_corr_cutoff = []
metrics_not_in_rocklin_scores = []
metrics_with_corr_NaN = []
ignore_metrics = ['sequence', 'description', 'dssp']
for metric in test_scores:
    
    # Skip over certain non-quantitative metrics
    if metric in ignore_metrics:
        continue
    
    # Make sure the Rocklin data has the metric
    if metric not in rocklin_scores:
        metrics_not_in_rocklin_scores.append(metric)
        continue
    
    # Correlate our score and the Rocklin score and store
    # in a dictionary. If R = 'NaN', don't store in dict
    (r,p) = scipy.stats.pearsonr(test_scores.loc[pdbs][metric],
                                 rocklin_scores.loc[pdbs][metric])
    if numpy.isnan(r):
        metrics_with_corr_NaN.append(metric)
        continue
    assert metric not in corr_d
    corr_d[metric] = {'Pearson_R':r}
    if r < corr_cutoff:
        metrics_below_corr_cutoff.append(metric)

# Show results
print("Compared the scores from {0} metrics on {1} designs".format(
                                                    len(corr_d),
                                                    len(pdbs)))
print("{0} metrics had R > {1}".format(
                                len(corr_d)-len(metrics_below_corr_cutoff),
                                corr_cutoff))
print("{0} metrics had R < {1}. They are: {2}".format(
                                len(metrics_below_corr_cutoff),
                                corr_cutoff,
                                ', '.join(metrics_below_corr_cutoff)))
print("{0} metrics had R = nan. They are: {1}".format(
                                len(metrics_with_corr_NaN),
                                ', '.join(metrics_with_corr_NaN)))
print("{0} metrics were not in the Rocklin scores: {1}".format(
                                len(metrics_not_in_rocklin_scores),
                                ', '.join(metrics_not_in_rocklin_scores)))

# Show a dataframe of each metric and its correlation coefficients
corr_df = pandas.DataFrame.from_dict(corr_d, orient="index")
corr_df

Next, I show correlation plots for each metric.

In [ ]:
metrics_to_plot = list(corr_d.keys()) + metrics_with_corr_NaN
nsubplots = len(metrics_to_plot)
ncols = 4.0
nrows = math.ceil(nsubplots/ncols)

sns.set(font_scale=1.5)
fig = plt.figure(figsize=[15,(7.5/2)*nrows])
for (i, metric) in enumerate(metrics_to_plot, 1):
    ax = fig.add_subplot(nrows, ncols, i)
    xs = test_scores.loc[pdbs][metric]
    ys = rocklin_scores.loc[pdbs][metric]
    sns.regplot(xs, ys, ax=ax)
    min_val = min(list(xs) + list(ys))
    max_val = max(list(xs) + list(ys))
    ticks = [min_val, max_val]
    lims = [min_val-abs(0.1*min_val), max_val+abs(0.1*max_val)]
    ax.axis('square')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    ax.set_xlabel('Our scores')
    ax.set_ylabel('Rocklin scores')
    ax.set_title(metric)
    #ax.text(0, 1, "R = {0}".format(round(corr_d[metric]['Pearson_R'], 2)))
plt.tight_layout()